In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import os
from reportlab.pdfbase.pdfmetrics import stringWidth


input_root_dir = 'txt/hofreps'
output_root_dir = 'pdf/hofreps'

def wrap_text(text, available_width, font_name, font_size):
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        test_line = f"{current_line} {word}".strip()
        if stringWidth(test_line, font_name, font_size) <= available_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)
    return lines

def convert_text_to_pdf(input_root_dir, output_root_dir):
    for root, dirs, files in os.walk(input_root_dir):
        for filename in files:
            if filename.endswith('.txt'):
                try:
                    input_file_path = os.path.join(root, filename)
                    relative_path = os.path.relpath(root, input_root_dir)
                    output_dir = os.path.join(output_root_dir, relative_path)
                    os.makedirs(output_dir, exist_ok=True)
                    output_file_path = os.path.join(output_dir, filename.replace('.txt', '.pdf'))
                    
                    c = canvas.Canvas(output_file_path, pagesize=letter)
                    width, height = letter
                    margin = 72
                    current_height = height - margin
                    line_height = 14
                    
                    available_width = width - 2 * margin
                    
                    with open(input_file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()
                        
                        # Set font for the title
                        c.setFont("Helvetica", 10)
                        for line in wrap_text(lines[0].strip(), available_width, "Helvetica", 10):
                            if current_height - line_height * 2 < margin:
                                c.showPage()
                                current_height = height - margin
                                c.setFont("Helvetica", 10)  # Reset font after creating a new page
                            c.drawString(margin, current_height, line)
                            current_height -= line_height * 2
                        
                        # Set font for the rest of the text
                        c.setFont("Helvetica", 10)
                        for line in lines[1:]:
                            wrapped_lines = wrap_text(line.strip(), available_width, "Helvetica", 10)
                            for wrapped_line in wrapped_lines:
                                if current_height - line_height < margin:
                                    c.showPage()
                                    current_height = height - margin
                                    c.setFont("Helvetica", 10)  # Reset font after creating a new page
                                c.drawString(margin, current_height, wrapped_line)
                                current_height -= line_height
                    
                    c.save()
                    print(f"Converted {input_file_path} to {output_file_path}")
                except Exception as e:
                    print(f"Failed to convert {filename} due to error: {e}")


convert_text_to_pdf(input_root_dir, output_root_dir)

Converted txt/hofreps/1.July01-July26-1901/10-Tuesday, 16th July, 1901.txt to pdf/hofreps/1.July01-July26-1901/10-Tuesday, 16th July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/13-Tuesday, 23rd July, 1901.txt to pdf/hofreps/1.July01-July26-1901/13-Tuesday, 23rd July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/7-Wednesday, 10th July, 1901.txt to pdf/hofreps/1.July01-July26-1901/7-Wednesday, 10th July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/4-Thursday, 4th July, 1901.txt to pdf/hofreps/1.July01-July26-1901/4-Thursday, 4th July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/1-Monday, 1st July, 1901.txt to pdf/hofreps/1.July01-July26-1901/1-Monday, 1st July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/3-Wednesday, 3rd July, 1901.txt to pdf/hofreps/1.July01-July26-1901/3-Wednesday, 3rd July, 1901.pdf
Converted txt/hofreps/1.July01-July26-1901/15-Thursday, 25th July, 1901.txt to pdf/hofreps/1.July01-July26-1901/15-Thursday, 25th July, 1901.pdf
Con